In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = tf.keras.utils.image_dataset_from_directory('Train_', image_size=(224,224), shuffle=True, batch_size=128)

Found 2100 files belonging to 5 classes.


In [3]:
data.class_names

['Ak_', 'Ala_Idris_', 'Buzgulu_', 'Dimnit_', 'Nazli_']

In [4]:
one_hot_encoded_dataset = data.map(lambda x, y: (x/255, y))


In [5]:
len(one_hot_encoded_dataset)

17

In [6]:
train_size = 10
valid_size = 3
test_size = 4


In [7]:
train = one_hot_encoded_dataset.take(train_size)
val = one_hot_encoded_dataset.skip(train_size).take(valid_size)
test = one_hot_encoded_dataset.skip(train_size+valid_size).take(test_size)

In [8]:
from tensorflow.keras.layers import Conv2D , MaxPooling2D , InputLayer,Flatten , GlobalAveragePooling2D ,AveragePooling2D, Dense , Dropout ,Activation , BatchNormalization, RandomFlip,  RandomRotation
from tensorflow.keras.models import Sequential

In [9]:
import tensorflow as tf

def inception_module(x, filters):
    # 1x1 
    conv1x1 = tf.keras.layers.Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    # 3x3 
    conv3x3 = tf.keras.layers.Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    conv3x3 = tf.keras.layers.Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3)

    # 5x5 
    conv5x5 = tf.keras.layers.Conv2D(filters[3], (1, 1), padding='same', activation='relu')(x)
    conv5x5 = tf.keras.layers.Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5x5)

    # pooling
    maxpool = tf.keras.layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    maxpool = tf.keras.layers.Conv2D(filters[5], (1, 1), padding='same', activation='relu')(maxpool)

    # Concatenate filters
    return tf.keras.layers.Concatenate(axis=-1)([conv1x1, conv3x3, conv5x5, maxpool])

def googlenet():
    inputs = tf.keras.Input(shape=(224, 224, 3))

    # Initial convolutions
    x = tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    # Inception modules
    x = inception_module(x, [64, 96, 128, 16, 32, 32])
    x = inception_module(x, [128, 128, 192, 32, 96, 64])
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [192, 96, 208, 16, 48, 64])
    x = inception_module(x, [160, 112, 224, 24, 64, 64])
    x = inception_module(x, [128, 128, 256, 24, 64, 64])
    x = inception_module(x, [112, 144, 288, 32, 64, 64])
    x = inception_module(x, [256, 160, 320, 32, 128, 128])
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [256, 160, 320, 32, 128, 128])
    x = inception_module(x, [384, 192, 384, 48, 128, 128])

    # Global average pooling
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # Output layer
    outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

# Instantiate the model
model = googlenet()
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 56, 56,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 56, 56,    │        256 │ max_pooling2d[0]… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │      6,240 │ batch_normalizat… │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │      1,040 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 56, 56,    │      4,160 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 56, 56,    │    110,720 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 56, 56,    │     12,832 │ conv2d_4[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 56, 56,    │      2,080 │ max_pooling2d_1[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 56, 56,    │          0 │ conv2d_1[0][0],   │
│ (Concatenate)       │ 256)              │            │ conv2d_3[0][0],   │
│                     │                   │            │ conv2d_5[0][0],   │
│                     │                   │            │ conv2d_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 56, 56,    │     32,896 │ concatenate[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 56, 56,    │      8,224 │ concatenate[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 56, 56,    │          0 │ concatenate[0][0] │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 56, 56,    │     32,896 │ concatenate[0][0] │
│                     │ 128)              │            │                 

 Total params: 5,837,365 (22.27 MB)

 Trainable params: 5,837,237 (22.27 MB)

 Non-trainable params: 128 (512.00 B)

In [10]:
model.fit(train, epochs=30, validation_data=val)

Epoch 1/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 216s 17s/step - accuracy: 0.2109 - loss: 1.9734 - val_accuracy: 0.1849 - val_loss: 1.6103
Epoch 2/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 177s 17s/step - accuracy: 0.1986 - loss: 1.6094 - val_accuracy: 0.1823 - val_loss: 1.6106
Epoch 3/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 201s 20s/step - accuracy: 0.2009 - loss: 1.6094 - val_accuracy: 0.2135 - val_loss: 1.6093
Epoch 4/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 261s 24s/step - accuracy: 0.2033 - loss: 1.6104 - val_accuracy: 0.1745 - val_loss: 1.6098
Epoch 5/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 278s 26s/step - accuracy: 0.1971 - loss: 1.6097 - val_accuracy: 0.2214 - val_loss: 1.6093
Epoch 6/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 234s 22s/step - accuracy: 0.1832 - loss: 1.6107 - val_accuracy: 0.2005 - val_loss: 1.6093
Epoch 7/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 221s 22s/step - accuracy: 0.2264 - loss: 1.6091 - val_accuracy: 0.2292 - val_loss: 1.6091
Epoch 8/30
10/10 ━━━━━━━━━━━━━━━━━━━━ 215s 21s/step - accuracy: 0.1957 - loss: 1.6095 - val_accuracy: 0.

In [11]:
from tensorflow.keras.metrics import Precision, Recall, SparseCategoricalAccuracy
pre = Precision()
re = Recall()
acc = SparseCategoricalAccuracy()

In [12]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    yhat_classes = np.argmax(yhat, axis=-1)
    
    
    pre.update_state(y, yhat_classes)
    re.update_state(y, yhat_classes)
    acc.update_state(y, yhat_classes)


4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step


In [13]:
print(pre.result(), re.result(), acc.result())

tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(0.20412844, shape=(), dtype=float32)


In [14]:
predictions = []
def get_pred(yhat):
    predictions.append(np.argmax(yhat))


In [15]:
import cv2
import os


path = 'Test/'

images = os.listdir(path)
# print(images)
for image in images:
    full_path = path + image
    img = cv2.imread(full_path)
#     print(img)
    img = img/255
    resized_img = tf.image.resize(img, (224, 224))
    yhat = model.predict(np.expand_dims(resized_img, 0))
    label = get_pred(yhat)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━

In [16]:
for i in range(len(images)):
    images[i] = images[i][:-4]

In [17]:
print(len(predictions))
print(len(images))

150
150


In [18]:
import pandas as pd

df = pd.DataFrame({'ID' : images, 'label': predictions})

In [19]:
df.head()

,ID,label
0,image1,0
1,image10,0
2,image100,0
3,image101,0
4,image102,0


In [20]:
df.to_csv('Custom_Model_predictions.csv', sep=',', index=False, encoding='utf-8')
